In [155]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [156]:
# 데이터 생성 (간단한 1차원 데이터)
def generate_real_data(samples):
    X1 = np.random.normal(0, 1, samples)
    y = np.ones((samples, 1))
    return X1, y

In [157]:
def generate_fake_data(generator, samples, latent_dim):
    noise = np.random.normal(0, 1, (samples, latent_dim))
    X = generator.predict(noise)
    y = np.zeros((samples, 1))
    return X, y

In [158]:
# Binary Cross-Entropy loss function
def bce_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred)

In [159]:
# Generator model
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(15, activation='relu', input_shape=(latent_dim,)))
    model.add(layers.Dense(1, activation='linear'))
    return model

In [160]:
# Discriminator model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(15, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [161]:
# Training function for BCE loss
def train_bce(generator, discriminator, combined, epochs, batch_size, latent_dim):
    for epoch in range(epochs):
        for _ in range(batch_size):
            # Train discriminator
            X_real, y_real = generate_real_data(batch_size)
            X_fake, y_fake = generate_fake_data(generator, batch_size, latent_dim)

            d_loss_real = discriminator.train_on_batch(X_real, y_real)
            d_loss_fake = discriminator.train_on_batch(X_fake, y_fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = combined.train_on_batch(noise, y_real)

        print(f"BCE Epoch {epoch + 1}/{epochs} [D loss: {d_loss:.6f}] [G loss: {g_loss:.6f}]")


In [162]:
# 주요 매개변수 설정
latent_dim = 5
epochs = 100
batch_size = 32

In [163]:
# Generator와 Discriminator 모델 빌드
generator_bce = build_generator(latent_dim)
discriminator_bce = build_discriminator()
discriminator_bce.compile(loss=bce_loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002))

# Combine 모델 빌드
discriminator_bce.trainable = False
z = tf.keras.Input(shape=(latent_dim,))
fake_data = generator_bce(z)
validity = discriminator_bce(fake_data)
combined_bce = tf.keras.Model(z, validity)
combined_bce.compile(loss=bce_loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002))
discriminator_bce.trainable = True

# GAN 학습 (BCE Loss)
train_bce(generator_bce, discriminator_bce, combined_bce, epochs, batch_size, latent_dim)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━

TypeError: unsupported format string passed to list.__format__

In [ ]:
# Wasserstein loss function
def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

In [ ]:
# Generator model
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, activation='relu', input_shape=(100,)))
    model.add(layers.Dense(28*28, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

In [ ]:
# Discriminator model
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1))
    return model

In [ ]:
# Load and preprocess the MNIST dataset
def load_data():
    (train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
    train_images = train_images / 127.5 - 1.0  # Normalize to [-1, 1]
    train_images = np.expand_dims(train_images, axis=-1)
    return train_images

In [ ]:
# Training function for Wasserstein loss
def train_wasserstein(generator, discriminator, combined, epochs, batch_size, latent_dim, data):
    d_losses = []
    g_losses = []
    for epoch in range(epochs):
        for batch in range(data.shape[0] // batch_size):
            # Train discriminator
            real_data = data[np.random.randint(0, data.shape[0], batch_size)]
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_data = generator.predict(noise)

            d_loss_real = discriminator.train_on_batch(real_data, -np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(fake_data, np.ones((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = combined.train_on_batch(noise, -np.ones((batch_size, 1)))

        d_losses.append(d_loss)
        g_losses.append(g_loss)
        if (epoch + 1) % 100 == 0:
            print(f"Wasserstein Epoch {epoch + 1}/{epochs}, Batch {batch + 1}/{data.shape[0] // batch_size} [D loss: {d_loss:.6f}] [G loss: {g_loss:.6f}]")
            save_images(generator, epoch + 1, latent_dim)

    return d_losses, g_losses

In [ ]:
# Function to save generated images
def save_images(generator, epoch, latent_dim, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'gan_generated_image_wasserstein_epoch_{epoch}.png')
    plt.close()

In [ ]:
latent_dim = 100
epochs = 1000
batch_size = 64

In [ ]:
data = load_data()

In [ ]:
# Build and compile models for Wasserstein loss
generator_wasserstein = build_generator()
discriminator_wasserstein = build_discriminator()
discriminator_wasserstein.compile(loss=wasserstein_loss, optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00005))

In [ ]:
# Combine models
discriminator_wasserstein.trainable = False
z = tf.keras.Input(shape=(latent_dim,))
fake_data = generator_wasserstein(z)
validity = discriminator_wasserstein(fake_data)
combined_wasserstein = tf.keras.Model(z, validity)
combined_wasserstein.compile(loss=wasserstein_loss, optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00005))
discriminator_wasserstein.trainable = True

In [ ]:
# Train the GAN with Wasserstein loss
d_losses_wasserstein, g_losses_wasserstein = train_wasserstein(generator_wasserstein, discriminator_wasserstein, combined_wasserstein, epochs, batch_size, latent_dim, data)